In [83]:
import pandas as pd

#pd.read_csv('fragmenstein_placed_other3A.csv')
triplacements = pd.read_pickle('fragmenstein_placed_other3A.pkl.gz')
biplacements = pd.read_pickle('fragmenstein_placed_other2.pkl.gz')

In [53]:
hit_replacements = pd.read_pickle('fragmenstein_hit_replacements_other2.pkl.gz')

In [55]:
from rdkit import Chem

with Chem.SDMolSupplier('new_other_hits_fragments.sdf') as sdfh:
    hits = list(sdfh)

print(len(hits))

56


In [82]:
len(acceptables)

3751

In [56]:
import json

with open('weights.json') as fh:
    weights = json.load(fh)

In [57]:
all_placements['N_interactions_lost'][weights.keys()]

KeyError: "None of [Index(['N_rotatable_bonds', '∆∆G', 'interaction_uniqueness_metric',\n       'N_unconstrained_atoms', 'N_constrained_atoms', 'N_interactions',\n       'N_interactions_lost', 'max_hit_Tanimoto'],\n      dtype='object')] are in the [index]"

In [59]:
all_placements = pd.concat([biplacements, triplacements], ignore_index=True)
#acceptables['N_interactions_lost'] = all_placements['N_interactions_lost'].fillna(10)

In [60]:
from fragmenstein import Laboratory

Laboratory.score(all_placements, hit_replacements, 'combo', hits, weights)

In [61]:
acceptables = all_placements.loc[all_placements.outcome == 'acceptable'].sort_values('ad_hoc_penalty')

In [63]:
km = KModes(n_clusters=20, init='Huang', n_init=5, verbose=1)
clusters = km.fit_predict(acceptables[[col for col in acceptables.columns if isinstance(col, tuple)]])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 561, cost: 5164.0
Run 1, iteration: 2/100, moves: 36, cost: 5164.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 237, cost: 5288.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 700, cost: 5136.0
Run 3, iteration: 2/100, moves: 396, cost: 5136.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1293, cost: 5690.0
Run 4, iteration: 2/100, moves: 15, cost: 5690.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 863, cost: 4936.0
Run 5, iteration: 2/100, moves: 49, cost: 4936.0
Best run was number 5


In [64]:
acceptables['intxn_cluster'] = clusters

In [65]:
from collections import defaultdict


def should_keep(cluster_idx, max_in_cluster:int= 10, counts = defaultdict(int)):    
    counts[cluster_idx] += 1
    return counts[cluster_idx] <= max_in_cluster

acceptables = acceptables.drop_duplicates(['cluster', 'intxn_cluster'])
acceptables['keep'] = acceptables.intxn_cluster.apply(should_keep)
acceptables[['N_interactions', 'cluster', 'intxn_cluster', 'keep']]

/tmp/ipykernel_52541/3063493811.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acceptables['keep'] = acceptables.intxn_cluster.apply(should_keep)


,N_interactions,cluster,intxn_cluster,keep
2428,7,5200,13,True
7047,7,108,0,True
4636,7,685,0,True
3867,8,703,0,True
497,8,5846,4,True
...,...,...,...,...
1315,4,434,19,False
10381,1,1892,10,False
10379,1,1894,10,False
4056,1,4531,8,False


In [68]:
import plotly.express as px

fig = px.scatter(acceptables,
                 x='∆∆G', y='ad_hoc_penalty',
                color='intxn_cluster',
                )
fig.update_layout(xaxis = dict(autorange = "reversed"),
                  yaxis = dict(autorange = "reversed"))

In [80]:
method_name = 'Fragmenstein-offsite-probes'
target_name='XX01ZVNS2B'
top_n = 200

from rdkit import Chem
from typing import List
from gist_import import GistImporter

fmodule = GistImporter.from_github('https://raw.githubusercontent.com/matteoferla/Fragment-hit-follow-up-chemistry/main/followup/prep_fragalysis.py')
prep = fmodule['prep']
generate_header = fmodule['generate_header']


too_new = []

def fix(mol: Chem.Mol):
    assert isinstance(mol, Chem.Mol)
    assert mol.GetNumAtoms()
    mol.ClearComputedProps()
    for name in mol.GetPropNames():
        mol.ClearProp(name)

with pd.option_context('mode.chained_assignment', None):
    df = acceptables[acceptables['keep']]\
                   .rename(columns={c: ':'.join(map(str, c)) for c in acceptables.columns if isinstance(c, tuple)})\
                   .head(top_n) \
                   .reset_index().copy()
    
# list of str to str w/ comma-separator
df['ref_mols'] = df.hit_names.apply(lambda l: [v.replace(f'{target_name}-', '') for v in l])\
                             .apply(lambda l: [v for v in l  if v not in too_new])\
                             .apply(','.join)
df['ref_mols'] = 'x0051_0B'
df['washed_mol'] = df.minimized_mol.apply(fix)
df['name'] = df['name'].apply(lambda v: v.split('-D68EV3CPROA')[0])
# non str/float/ints
not_okay = ('name', 'minimized_mol', 'ref_mols', 'washed_mol', 'mode',
            'runtime','error','outcome',
            'smiles',
             'regarded', 'disregarded', 'hit_names',
              '∆G_bound',
             '∆G_unbound',
             'unmin_binary',
             'min_binary',
            'binary_hits', 'custom_map', 'PAINSes',
             'hit_binaries',
            'minimized_mol',
              'hit_mols','unminimized_mol', 'hit_names')
nonempty = [c for c, v in (df[[c for c in df.columns if ':' in c]].sum() > 0).to_dict().items() if v]
extras: List[str] = [c for c in df.columns if c not in not_okay and ':' not in c] + nonempty
def floatify(value):
    try:
        return float(value)
    except Exception:
        return float('nan')

bad_columns = [k for k in extras if df[k].apply(floatify).isna().any()]
assert not bad_columns, f'Some entries are not numeric {bad_columns}'

for col in extras:
    df[col] = df[col].apply(floatify)
    
header: Chem.Mol = generate_header(method_name,
                         ref_url='https://github.com/matteoferla/Fragment-hit-follow-up-chemistry',
                         submitter_name='Matteo Ferla',
                         submitter_email='matteo.ferla@stats.ox.ac.uk',
                         extras=dict(zip(extras, extras))
                        )
prep(df, header, mol_col='minimized_mol', name_col='name', outfile=f'{method_name}.sdf', ref_pdb='x0051_0B',
     extras=extras
    )
print(f'{method_name}.sdf')

Fragmenstein-offsite-probes.sdf


[12:54:24] Molecule does not have explicit Hs. Consider calling AddHs()


In [79]:
df

,index,smiles,name,binary_hits,custom_map,error,mode,∆∆G,∆G_bound,∆G_unbound,...,UFF_Gibbs,strain_per_HA,ad_hoc_penalty,cluster,intxn_cluster,keep,ref_mols,washed_mol,original SMILES,ref_pdb
0,2428,CC1=CC=CC=C1CNC(=O)NNC2=CC=CC=C2,Z1209702068-Z198195906S4-x0884S4,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'Z198195906S4': {0: 0, 1: 1, 2: 2, 3: 6, 4: 3...",,expansion,-8.145830,-7.752416,0.393414,...,111.441076,5.865289,-55.498710,5200,13,True,x0051_0B,None,[H]c1c([H])c([H])c(N([H])N([H])C(=O)N([H])C([H...,x0051_0B
1,7047,CC1CNCCN1CC2=CC=C(F)C(Br)=C2,Z2444712578-x0944S1-x0130S3,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0944S1': {0: 6, 1: 5, 2: 1, 3: 4, 4: 2, 5: ...",,expansion,-4.028487,-3.969379,0.059108,...,166.468809,10.404236,-48.918057,108,0,True,x0051_0B,None,[H]c1c([H])c(C([H])([H])N2C([H])([H])C([H])([H...,x0051_0B
2,4636,C[C@@H]1CNCCN1CC2=CC=CC(F)=C2,Z2444734753-x0944S1-x0944S3,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0944S1': {0: 6, 1: 5, 2: 1, 3: 4, 4: 2, 5: ...",,expansion,-3.835400,-4.093992,-0.258593,...,72.852395,4.856794,-46.762248,685,0,True,x0051_0B,None,[H]c1c([H])c(F)c([H])c(C([H])([H])N2C([H])([H]...,x0051_0B
3,3867,CC(C1=CC(F)=C(F)C(F)=C1)N2CCNC[C@@H]2C,Z4562665581-x0944-x0130S3,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0944': {0: 11, 1: 4, 2: 0, 3: 6, 4: 14, 5: ...",,expansion,-3.343394,-3.062131,0.281263,...,636696.803126,35371.848108,-44.648146,703,0,True,x0051_0B,None,[H]c1c(F)c(F)c(F)c([H])c1C([H])(N1C([H])([H])C...,x0051_0B
4,497,O=C(CC1=CC=CC2=CC=CC=C12)NC3=CC=CC=C3,Z27782615-Z1318055457S2-x0806S1,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'Z1318055457S2': {0: 2, 1: 3, 2: 4, 3: 12, 4:...",,expansion,-5.573028,-5.235797,0.337231,...,158.810826,7.940502,-41.601453,5846,4,True,x0051_0B,None,[H]c1c([H])c([H])c(N([H])C(=O)C([H])([H])c2c([...,x0051_0B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,8570,CC(NCC1=CN=CC(F)=C1)C(N)=O,Z1745075149-x0884S3-x0214S3,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0884S3': {0: 2, 1: 1, 2: 0, 3: 11, 4: 12, 5...",,expansion,-2.284428,-2.052511,0.231916,...,67.170331,4.797846,-4.177921,2645,9,True,x0051_0B,None,[H]c1nc([H])c(C([H])([H])N([H])C([H])(C(=O)N([...,x0051_0B
196,3394,CC1=CC(CNC(C)COC2=CC=CC=C2)=CC=C1,Z1164134152-x0884-Z198195906,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0884': {0: 8, 1: 0, 4: -1, 6: 15, 7: 14, 8:...",,expansion,-5.306529,-5.336885,-0.030356,...,1261.763771,66.408270,-3.854044,4752,11,True,x0051_0B,None,[H]c1c([H])c([H])c(OC([H])([H])C([H])(N([H])C(...,x0051_0B
197,8569,CC(NCC1=CN=CC(Cl)=C1)C(N)=O,Z2697260684-x0884S3-x0214S3,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'x0884S3': {0: 2, 1: 1, 2: 0, 3: 11, 4: 12, 5...",,expansion,-2.085607,-1.780861,0.304746,...,122.623541,8.758762,-3.581459,2646,9,True,x0051_0B,None,[H]c1nc([H])c(C([H])([H])N([H])C([H])(C(=O)N([...,x0051_0B
198,10128,NC(=O)CCC1=CC=CC=C1NC(=O)C(N)=O,PV-007078088111-Z198195906S1-x0884,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\...,"{'Z198195906S1': {0: 0, 1: 1, 2: 2, 3: 3}, 'x0...",,expansion,-3.416815,-1.386822,2.029993,...,119.891023,7.052372,-2.681069,1996,9,True,x0051_0B,None,[H]c1c([H])c([H])c(C([H])([H])C([H])([H])C(=O)...,x0051_0B


In [ ]:
conda create -n compchem -y python=3.11
conda activate compchem

conda install -y -n base -c conda-forge distro nodejs sqlite nb_conda_kernels
conda update -y -c conda-forge nodejs   # peace among worlds

python -m pip install -q pandas plotly seaborn pillow pandas pandarallel pandera nglview pebble rdkit jupyterlab-spellchecker;
conda install -y -n base -c conda-forge openssh nano;
conda install -y -n base -c conda-forge util-linux;
conda install -y -c conda-forge htop;
conda install -y -n base -c conda-forge openbabel plip git;
conda install -y -n base -c conda-forge -c bioconda kalign2 hhsuite muscle hhsuite mmseqs2;
python -m pip install -q fragmenstein  pyrosetta_help
python -m pip install -q $DATA/shared/pyrosetta-2023.27+release.e3ce6ea9faf-cp311-cp311-linux_x86_64.whl
conda install -y  -c nvidia -c conda-forge cuda-toolkit=11.3.0 cuda-nvcc cuda-command-line-tools gputil
conda install -y -c omnia -c conda-forge openmm openff-forcefields openff-toolkit openmmforcefields
conda install -y -c pytorch -c conda-forge pytorch torchvision matplotlib pandas 

In [ ]:
packages/fragmenstein/laboratory/_combine.py", line 72, in combine
    df['simple_smiles'] = df.unminimized_mol.apply(self.Victor.to_simple_smiles)



In [6]:
from rdkit import Chem

with Chem.SDMolSupplier('XX01ZVNS2B.corrected.sdf') as sdfh:
    hits = list(sdfh)

In [7]:
hitdex = {h.GetProp('_Name'): h for h in hits}

In [9]:
for k in 'x0182 x0806 x0214 x0884'.split():
    for kh in hitdex.keys():
        if k in kh:
            print(kh)

x0182_0B
x0182_1B
x0884_0B
x0884_1B


In [10]:
500 / 12

41.666666666666664

In [11]:
import pymol2

ModuleNotFoundError: No module named 'pymol2'

In [1]:
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
mol = Chem.MolFromSmiles('CCSCC')
SimilarityMaps.GetMorganFingerprint(mol, fpType='normal')

ValueError: Unknown Morgan fingerprint type